# Read Data From CSV

In [1]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from numpy.linalg import norm
from quaternion import Quaternion

In [2]:
#функции калибровки
def find_offsets(mpu,counts = 85):
#     calibration_set = mpu.loc[:counts]
    calibration_set =mpu 
    n = float(len(calibration_set))
    summ = []
    offsets = []
    for i in calibration_set:
        summ.append(calibration_set[i].sum())
    for s in summ:
        offsets.append(s/n)
    return offsets

def find_headers(mpu):
    headers = []
    for i in mpu:
        headers.append(i)
    return headers

def calibration(mpu):
    
    headers = find_headers(mpu)
    print(headers)
    
    off = find_offsets(mpu)
    calibrated = mpu.copy()
    #не калибровать ускорение для фильтра маджвика!
    calibrated[headers[0]] = mpu[headers[0]] - off[0]
#     calibrated[headers[1]] = mpu[headers[1]] - off[1]
    calibrated[headers[2]] = mpu[headers[2]] - off[2]
    
    calibrated[headers[3]] = mpu[headers[3]] - off[3]
    calibrated[headers[4]] = mpu[headers[4]] - off[4]
    calibrated[headers[5]] = mpu[headers[5]] - off[5]
    
    return calibrated


In [53]:
def filter_outliers(df_local):
    df = df_local.copy()
#     o = ['rt_gx']
    median = df['rt_gx'].median()
    std = df['rt_gx'].std()
    df.loc[(df['rt_gx'] - median).abs() > std,'rt_gx']=np.nan
    df['rt_gx'].fillna(median, inplace=True)
    return df

def find_outliers(df):


    rt_outl =filter_outliers(df)
    print(rt_outl)

    return rt_outl

In [54]:
#функции для вывода графиков
def show_plots(mpu_t,mpu_b,title_t,title_b,accel=None):
    
    headers_t = find_headers(mpu_t)
    headers_b = find_headers(mpu_b)
    
    if accel == True:
        headers_t = headers_t[:3]
        headers_b = headers_b[:3]
        title_t += ' accel'
        title_b += ' accel'
    elif accel == None:
        headers_t = headers_t[:3]
        title_t +='accel'
        headers_b = headers_b[3:]
        title_b += ' gyro'
    else:
        headers_t = headers_t[3:]
        headers_b = headers_b[3:]
        title_t += ' gyro'
        title_b += ' gyro'
        


    fig = make_subplots(rows=1, cols=2, subplot_titles=(title_t,title_b))


    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[0]],name=headers_t[0]),1,1)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[1]],name=headers_t[1]),1,1)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[2]],name=headers_t[2]),1,1)

    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_b[headers_b[0]],name=headers_b[0]),1,2)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_b[headers_b[1]],name=headers_b[1]),1,2)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_b[headers_b[2]],name=headers_b[2]),1,2)


    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.5, xanchor="center"),
                      hovermode="x",
                      margin=dict(l=0, r=0, t=40, b=0))
    fig.show()


def show_plot(mpu_t,title_t,accel=None):
    
    headers_t = find_headers(mpu_t)
    
    if accel == True:
        headers_t = headers_t[:3]
        title_t += ' accel'
    elif accel == None:
        headers_t = headers_t
    else:
        headers_t = headers_t[3:]
        title_t += ' gyro'
        


    fig = go.Figure() 
    fig.update_layout(title=title_t)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[0]],name=headers_t[0]))
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[1]],name=headers_t[1]))
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[2]],name=headers_t[2]))

    fig.show()


In [55]:
df = pd.read_csv('20_all_legs.csv',sep = ' ',header = None)
df.columns = ["s1","bt_ax","bt_ay","bt_az","bt_gx","bt_gy","bt_gz",
            "s2","bb_ax","bb_ay","bb_az","bb_gx","bb_gy","bb_gz",
            "s3","lt_ax","lt_ay","lt_az","lt_gx","lt_gy","lt_gz",
            "s4","lb_ax","lb_ay","lb_az","lb_gx","lb_gy","lb_gz",
            "s5","rt_ax","rt_ay","rt_az","rt_gx","rt_gy","rt_gz",
             "s6","rb_ax","rb_ay","rb_az","rb_gx","rb_gy","rb_gz",
             ]
# df

In [57]:
#each  IMU for calibration
bt = df[["bt_ax","bt_ay","bt_az","bt_gx","bt_gy","bt_gz"]]
bb = df[["bb_ax","bb_ay","bb_az","bb_gx","bb_gy","bb_gz"]]
lt = df[["lt_ax","lt_ay","lt_az","lt_gx","lt_gy","lt_gz"]]
lb = df[["lb_ax","lb_ay","lb_az","lb_gx","lb_gy","lb_gz"]]
rt = df[["rt_ax","rt_ay","rt_az","rt_gx","rt_gy","rt_gz"]]
rb = df[["rb_ax","rb_ay","rb_az","rb_gx","rb_gy","rb_gz"]]
rt_outl  = filter_outliers(rt)

In [58]:
show_plot(bt,'Non calibrated back-top ',False)
show_plot(bb,'Non calibrated back-bottom ',False)

In [59]:
show_plot(lt,'left- top ',False)
show_plot(lb,'Non calibrated left bottom ',False)

In [61]:
show_plot(rt,'Non calibrated right-top ',False)
# show_plot(lt,'Non calibrated left-top ',False)

show_plot(rt_outl,'Non calibrated right-top ',False)


 Calibrate DF of each sensor

In [70]:
rt= rt_outl
calibrated_bt = calibration(bt)
calibrated_bb = calibration(bb)
calibrated_lt = calibration(lt)
calibrated_lb = calibration(lb)
calibrated_rt = calibration(rt)
calibrated_rb = calibration(rb)

# calibrated_bt = bt
# calibrated_bb = bb
# calibrated_lt = lt
# calibrated_lb = lb
# calibrated_rt = rt
# calibrated_rb = rb


['bt_ax', 'bt_ay', 'bt_az', 'bt_gx', 'bt_gy', 'bt_gz']
['bb_ax', 'bb_ay', 'bb_az', 'bb_gx', 'bb_gy', 'bb_gz']
['lt_ax', 'lt_ay', 'lt_az', 'lt_gx', 'lt_gy', 'lt_gz']
['lb_ax', 'lb_ay', 'lb_az', 'lb_gx', 'lb_gy', 'lb_gz']
['rt_ax', 'rt_ay', 'rt_az', 'rt_gx', 'rt_gy', 'rt_gz']
['rb_ax', 'rb_ay', 'rb_az', 'rb_gx', 'rb_gy', 'rb_gz']


In [72]:
title_t = 'Non-calibrated left-top sensor '
title_b = 'Calibrated left-top sensor '

show_plots(rt,calibrated_rt,title_t,title_b,False)

# find orientation

In [73]:
import madgwickahrs as mg
from quaternion import Quaternion

In [74]:
#find only with gyro 

pitch = np.zeros(len(calibrated_lb))
yaw = np.zeros(len(calibrated_lb))
roll = np.zeros(len(calibrated_lb))

for i in range(len(calibrated_lb)-1):
    delta_t = 0.06
    
    pitch[i+1] = pitch[i] + calibrated_lb['lb_gy'][i]*delta_t
    yaw[i+1] = yaw[i]+calibrated_lb['lb_gz'][i]*delta_t
    roll[i+1] = roll[i]+calibrated_lb['lb_gx'][i]*delta_t
    

orientation_d = pd.DataFrame({'roll':np.rad2deg(roll),'pitch':np.rad2deg(pitch),'yaw':np.rad2deg(yaw)})

show_plot(orientation_d,'Orientation only with  gyro')


In [75]:
# Madgwick filter

In [76]:
def find_orientation(calibrated_mpu):
    
    headers = find_headers(calibrated_mpu)
        
    quaternions = []

    heading = mg.MadgwickAHRS(sampleperiod = 0,beta = 1)
    for i,row in calibrated_mpu.iterrows():
        gyro_d  = []
        accel_d = []
        gyro_d.append(row[headers[3]])
        gyro_d.append(row[headers[4]])
        gyro_d.append(row[headers[5]])
        accel_d.append(row[headers[0]])
        accel_d.append(row[headers[1]])
        accel_d.append(row[headers[2]])

        heading.update_imu(gyro_d,accel_d)
        quaternions.append(heading.quaternion._get_q())

    # print(quaternions)
    pitch = []
    roll = []
    yaw = []
    for i in range(len(quaternions)):
        pitch.append(np.arcsin(2 * quaternions[i][1] * quaternions[i][2] + 2 * quaternions[i][0] * quaternions[i][3]))
        if np.abs(quaternions[i][1] * quaternions[i][2] + quaternions[i][3] * quaternions[i][0] - 0.5) < 1e-8:
            roll.append(0) 
            yaw.append(2 * np.arctan2(quaternions[i][1], quaternions[i][0]))
        elif np.abs(quaternions[i][1] * quaternions[i][2] + quaternions[i][3] * quaternions[i][0] + 0.5) < 1e-8:
            roll.append(-2 * np.arctan2(quaternions[i][1], quaternions[i][0]))
            yaw.append(0)
        else:
            roll.append(np.arctan2(2 * quaternions[i][0] * quaternions[i][1] - 2 * quaternions[i][2] *quaternions[i][3], 1 - 2 * quaternions[i][1] ** 2 - 2 *quaternions[i][3] ** 2))
            yaw.append(np.arctan2(2 * quaternions[i][0] * quaternions[i][2] - 2 * quaternions[i][1] * quaternions[i][3], 1 - 2 * quaternions[i][2] ** 2 - 2 * quaternions[i][3] ** 2))


    orientation_d = pd.DataFrame({'roll':np.rad2deg(roll),'pitch':np.rad2deg(pitch),'yaw':np.rad2deg(yaw)})
    return orientation_d

In [78]:
orientation_lb = find_orientation(calibrated_rt)
show_plot(orientation_lb,'Orientation left_bot sensor')

In [19]:


# quaternions = []

# heading = mg.MadgwickAHRS(sampleperiod = 0.06,beta = 1)
# for i,row in calibrated_lb.iterrows():
#     gyro_d  = []
#     accel_d = []
#     gyro_d.append(row['lb_gx'])
#     gyro_d.append(row['lb_gy'])
#     gyro_d.append(row['lb_gz'])
#     accel_d.append(row['lb_ax'])
#     accel_d.append(row['lb_ay'])
#     accel_d.append(row['lb_az'])

#     heading.update_imu(gyro_d,accel_d)
#     quaternions.append(heading.quaternion._get_q())

# # print(quaternions)
# pitch = []
# roll = []
# yaw = []
# for i in range(len(quaternions)):
#     pitch.append(np.arcsin(2 * quaternions[i][1] * quaternions[i][2] + 2 * quaternions[i][0] * quaternions[i][3]))
#     if np.abs(quaternions[i][1] * quaternions[i][2] + quaternions[i][3] * quaternions[i][0] - 0.5) < 1e-8:
#         roll.append(0) 
#         yaw.append(2 * np.arctan2(quaternions[i][1], quaternions[i][0]))
#         print('thth')
#     elif np.abs(quaternions[i][1] * quaternions[i][2] + quaternions[i][3] * quaternions[i][0] + 0.5) < 1e-8:
#         roll.append(-2 * np.arctan2(quaternions[i][1], quaternions[i][0]))
#         yaw.append(0)
#     else:
#         roll.append(np.arctan2(2 * quaternions[i][0] * quaternions[i][1] - 2 * quaternions[i][2] *quaternions[i][3], 1 - 2 * quaternions[i][1] ** 2 - 2 *quaternions[i][3] ** 2))
#         yaw.append(np.arctan2(2 * quaternions[i][0] * quaternions[i][2] - 2 * quaternions[i][1] * quaternions[i][3], 1 - 2 * quaternions[i][2] ** 2 - 2 * quaternions[i][3] ** 2))


# orientation_d = pd.DataFrame({'roll':roll,'pitch':pitch,'yaw':yaw})

In [20]:
# orientation_d

In [21]:
# show_plot(lb,'gyro_lb',False)

In [22]:
# fig = go.Figure() 
# fig.update_layout(title='roll lb')

# fig.add_trace(go.Scatter(x=orientation_d.index, y=np.rad2deg(orientation_d['roll']),name='roll'))
# fig.add_trace(go.Scatter(x=orientation_d.index, y=np.rad2deg(orientation_d['pitch']),name='pitch'))
# fig.add_trace(go.Scatter(x=orientation_d.index, y=np.rad2deg(orientation_d['yaw']),name='yaw'))
# # fig.add_trace(go.Scatter(x=agles.index, y=agles['x'],name='gyro'))

# fig.show()

In [ ]:
#(-0.02213089742574257, 0.01005698124752475, -0.013309934386138617)